<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Reservoir%20Simulation%20Ertekin/Unit%205%20Finite-Difference%20Approximation%20to%20Linear-Flow%20Equations/fd1d_linearflow_implicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 5 Finite-Difference Approximations to Linear-Flow Problems**

## Implicit Formulation (*Backward-Difference Approximation*)

![Fig 5 30 Implicit Method](https://user-images.githubusercontent.com/51282928/76113827-60a28780-6017-11ea-9646-bb1c66683c08.PNG)

$$T_{lx_{i+1/2}}^{n} p_{i+1}^{n+1}-[(\frac{V_b \phi c_l}{\alpha_c B_l \Delta t})_{i} + T_{lx_{i+1/2}}^{n}+T_{lx_{i-1/2}}] \cdot p_{i}^{n+1} + T_{lx_{i-1/2}}^{n} p_{i-1}^{n+1} = -[q_{lsc_i}+(\frac{V_b \phi c_l}{\alpha_c B_l \Delta t})_{i} \cdot p_{i}^{n}]$$

In the following codes, $(\frac{V_b \phi c_l}{\alpha_c B_l \Delta t})_{i}$ will be written as `factor`

Transmissibility of coupling cells, $T_{lx_{i\pm1/2}}^{n}$, written as `T_min_half` or `T_plus_half`

The above equation will be simplified to:

$$pi_{next}^{+} \cdot p_{i+1}^{n+1}-pi_{next} \cdot p_{i}^{n+1} + pi_{next}^{-} \cdot p_{i-1}^{n+1} = pi$$

Where $pi_{next}^{+}$ (`pi_next_plus`), $pi_{next}$ (`pi_next`), $pi_{next}^{-}$ (`pi_next_minus`), and $pi$ (`pi`) are coefficients that contain the above `T_min_half` and `T_plus_half`.



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 922 (delta 5), reused 0 (delta 0), pack-reused 909
Receiving objects: 100% (922/922), 12.07 MiB | 37.56 MiB/s, done.
Resolving deltas: 100% (400/400), done.


## Example 5.11 Five reservoir gridblocks with no flow at both boundaries

![Example 5 8](https://user-images.githubusercontent.com/51282928/75949840-09e36380-5eda-11ea-824c-dd5c9ac6f6d5.PNG)

### For timestep 15 days

In [12]:
"Task. determine pressure distribution during first year of production with timestep 10 days"

# known
p_initial = 6000 # initial pressure, in psia
delta_x = 1000 # ft
delta_y = 1000
delta_z = 75
ngrid = 5
grid_loc = 4 # grid location where production well is located
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 3.5E-06 # phase compressibility, psi^-1
k_x = 15 # perm in x direction, md
poro = 0.18
mu = 10 # phase viscosity, cp

delta_t = 15 # days
qsc = -150 # minus means production, stb/d

# conversion
k_x = 15 * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (Vb * poro * c) / (alpha * B * delta_t)
factor

0.5609973285841495

Because of uniform gridblocks, the equation for `tr_plus_coupling` and `tr_min_coupling` become:

$$T_{lx_{i+1/2}}^{n} = T_{lx_{i-1/2}}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i+1/2}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i-1/2}^{n}$$

In [13]:
# calculate transmissibility of coupling cells
beta = 1.127 # transmissibility conversion factor, is a constant
Ax = delta_y * delta_z
T_plus_half = beta * ((Ax * k_x) / (mu * B * delta_x))
T_min_half = T_plus_half
T_min_half, T_plus_half

(0.1267875, 0.1267875)

There are 5 grids (`grid_1`, `grid_2`, `grid_3`, `grid_4`, `grid_5`), so the transmissibility coefficients will be: $$T_{lx_{i+1/2}}^{n}=(T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n}, T_{lx_{5+1/2}}^{n})$$

and

$$T_{lx_{i-1/2}}^{n}=(T_{lx_{1/2}}^{n}, T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n})$$

The values of $T_{lx_{5+1/2}}^{n}=0$ and $T_{lx_{1/2}}^{n}=0$ (the edge of gridblocks), so the values of each are:

$$T_{lx_{i+1/2}}^{n}=(0.127, 0.127, 0.127, 0.127, 0)$$

and

$$T_{lx_{i-1/2}}^{n}=(0, 0.127, 0.127, 0.127, 0.127)$$




In [14]:
q = np.full(ngrid-1, T_min_half)
Ti_plus_halves = np.append(q, [0])
print(Ti_plus_halves)
p = np.full(ngrid-1, T_plus_half)
Ti_min_halves = np.append([0], p)
print(Ti_min_halves)
print("At grid 1, the coupling transmissibility coeffs are:", Ti_min_halves[0], "for T_min_half and:", Ti_plus_halves[0], "for T_plus_half.")
print("At grid 3, the coupling transmissibility coeffs are:", Ti_min_halves[2], "for T_min_half and:", Ti_plus_halves[2], "for T_plus_half.")
print("At grid 5, the coupling transmissibility coeffs are:", Ti_min_halves[4], "for T_min_half and:", Ti_plus_halves[4], "for T_plus_half.")

[0.1267875 0.1267875 0.1267875 0.1267875 0.       ]
[0.        0.1267875 0.1267875 0.1267875 0.1267875]
At grid 1, the coupling transmissibility coeffs are: 0.0 for T_min_half and: 0.1267875 for T_plus_half.
At grid 3, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.1267875 for T_plus_half.
At grid 5, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.0 for T_plus_half.


Requires array for $q_{sc}$

In [0]:
qsc = [0, 0, 0, -150, 0] # production well in grid 4

Calculate $p_{i}^{n+1}$ for each grid in each time



### For the first timestep

After iteration for the **FIRST timestep**, there will be four coefficients computed: `pi_next_plus_arr`, `pi_next_arr`, `pi_next_minus_arr`, and `pi_arr` (as already explained above, each represents the coefficient of $p_{i+1}^{n+1}$, $p_{i}^{n+1}$, $p_{i-1}^{n+1}$, and $p_{i}^{n}$). The array size equals to number of grids `ngrid`.

* `pi_next_plus_arr` = coeff [$p_{2}^{n+1}, p_{3}^{n+1}, p_{4}^{n+1}, p_{5}^{n+1}, p_{6}^{n+1}$] = `[0.13, 0.13, 0.13, 0.13, 0]`; size = `ngrid`

* `pi_next_arr` = coeff [$p_{1}^{n+1}, p_{2}^{n+1}, p_{3}^{n+1}, p_{4}^{n+1}, p_{5}^{n+1}$] = `[-0.68, -0.81, -0.81, -0.81, -0.68]`; size = `ngrid`

* `pi_next_minus_arr` = coeff [$p_{0}^{n+1}, p_{1}^{n+1}, p_{2}^{n+1}, p_{3}^{n+1}, p_{4}^{n+1}$] = `[0, 0.13, 0.13, 0.13, 0.13]`; size = `ngrid`

* `pi_arr` = [$p_{1}^{n}, p_{2}^{n}, p_{3}^{n}, p_{4}^{n}, p_{5}^{n}$] = `[5999.08253536, 5995.02302466, 5968.94186457, 5805.43760672, 5964.13401632]`; size = `ngrid`

Then, $p_{0}^{n+1}$ and $p_{6}^{n+1}$ is **deleted** from the array because we want to create a **tridiagonal matrix**.

* `pi_next_plus_arr` = coeff [$p_{2}^{n+1}, p_{3}^{n+1}, p_{4}^{n+1}, p_{5}^{n+1}, p_{6}^{n+1}$] = `[0.13, 0.13, 0.13, 0.13]`; size = `ngrid-1`

* `pi_next_arr` = coeff [$p_{1}^{n+1}, p_{2}^{n+1}, p_{3}^{n+1}, p_{4}^{n+1}, p_{5}^{n+1}$] = `[-0.68, -0.81, -0.81, -0.81, -0.68]`; size = `ngrid`

* `pi_next_minus_arr` = coeff [$p_{0}^{n+1}, p_{1}^{n+1}, p_{2}^{n+1}, p_{3}^{n+1}, p_{4}^{n+1}$] = `[0.13, 0.13, 0.13, 0.13]`; size = `ngrid-1`

* `pi_arr` = [$p_{1}^{n}, p_{2}^{n}, p_{3}^{n}, p_{4}^{n}, p_{5}^{n}$] = `[5999.08253536, 5995.02302466, 5968.94186457, 5805.43760672, 5964.13401632]`; size = `ngrid`

$$\begin{bmatrix} coeff(p_{1}^{n}) & coeff(p_{2}^{n}) & 0 & 0 & 0 \\ coeff(p_{1}^{n}) & coeff(p_{2}^{n}) & coeff(p_{3}^{n}) & 0 & 0 \\ 0 & coeff(p_{2}^{n}) & coeff(p_{3}^{n}) & coeff(p_{4}^{n}) & 0 & \\ 0 & 0 & coeff(p_{3}^{n}) & coeff(p_{4}^{n}) & coeff(p_{5}^{n}) \\ 0 & 0 & 0 & coeff(p_{4}^{n}) & coeff(p_{5}^{n}) \end{bmatrix}$$

In numbers: 

$$A=\begin{bmatrix} -0.69 & 0.13 & 0 & 0 & 0 \\ 0.13 & -0.81 & 0.13 & 0 & 0 \\ 0 & 0.13 & -0.81 & 0.13 & 0 & \\ 0 & 0 & 0.13 & -0.81 & 0.13 \\ 0 & 0 & 0 & 0.13 & -0.69 \end{bmatrix}$$

We will solve for the five $p_{1}^{n+1}, p_{2}^{n+1}, p_{3}^{n+1}, p_{4}^{n+1}, p_{5}^{n+1}$ using TDMA.

$$A \cdot X = B$$

Where, $X=\begin{bmatrix} p_{1}^{n+1} \\ p_{2}^{n+1} \\ p_{3}^{n+1} \\ p_{4}^{n+1} \\ p_{5}^{n+1}\end{bmatrix}$, and $B=\begin{bmatrix} -3365.98 \\ -3365.98 \\ -3365.98 \\ -3215.98 \\ -3365.98 \end{bmatrix}$
 




In [16]:
import sys, os
sys.path.append('/content/reservoir-engineering/Reservoir Simulation Ertekin/Unit 5 Finite-Difference Approximation to Linear-Flow Equations')
from tdma_implicit import TDMAsolver

pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]

pi_next_plus_arr = []
pi_next_arr = []
pi_next_minus_arr = []
pi_arr = []

for i in range(len(pi)):
  pi_next_plus = Ti_plus_halves[i]
  pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
  pi_next_minus = Ti_min_halves[i]
  pi_current = -(qsc[i] + (factor * pi[i]))

  pi_next_plus_arr.append(float(pi_next_plus))
  pi_next_arr.append(float(pi_next))
  pi_next_minus_arr.append(float(pi_next_minus))
  pi_arr.append(float(pi_current))

pi_next_plus_arr = pi_next_plus_arr[:-1]
pi_next_minus_arr = pi_next_minus_arr[1:]

p_new = TDMAsolver(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr, pi_arr)
p_new

array([5999.08253536, 5995.02302466, 5968.94186457, 5805.43760672,
       5964.13401632])

In [17]:
# form of matrix
def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

# pi_ = [0.1268, 0.1268, 0.1268]; b = [-0.8146, -0.8146, -0.8146, -0.6878]; c = [0.1268, 0.1268, 0.1268]
A = tridiag(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr)
A

array([[-0.68778483,  0.1267875 ,  0.        ,  0.        ,  0.        ],
       [ 0.1267875 , -0.81457233,  0.1267875 ,  0.        ,  0.        ],
       [ 0.        ,  0.1267875 , -0.81457233,  0.1267875 ,  0.        ],
       [ 0.        ,  0.        ,  0.1267875 , -0.81457233,  0.1267875 ],
       [ 0.        ,  0.        ,  0.        ,  0.1267875 , -0.68778483]])

### For the next timesteps

At the **FIRST timestep**, the pressures has been solved: 

$$X=\begin{bmatrix} p_{1}^{n+1} \\ p_{2}^{n+1} \\ p_{3}^{n+1} \\ p_{4}^{n+1} \\ p_{5}^{n+1}\end{bmatrix} = \begin{bmatrix} 5999.08 \\ 5995.02 \\ 5968.94 \\ 5805.44 \\ 5964.13 \end{bmatrix}$$ 

The iteration continues to the **SECOND timestep**, where the pressure are **updated**.

$$\begin{bmatrix} p_{1}^{n} \\ p_{2}^{n} \\ p_{3}^{n} \\ p_{4}^{n} \\ p_{5}^{n}\end{bmatrix} = \begin{bmatrix} 5999.08 \\ 5995.02 \\ 5968.94 \\ 5805.44 \\ 5964.13 \end{bmatrix}$$ 

And then re-compute for $X$, et cetera for the **NEXT timesteps**

In [8]:
import sys, os
sys.path.append('/content/reservoir-engineering/Reservoir Simulation Ertekin/Unit 5 Finite-Difference Approximation to Linear-Flow Equations')
from tdma_implicit import TDMAsolver

pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

p_new_arr = []

for j in range(len(time)):
  pi_next_plus_arr = []
  pi_next_arr = []
  pi_next_minus_arr = []
  pi_arr = []

  for i in range(len(pi)):
    pi_next_plus = Ti_plus_halves[i]
    pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
    pi_next_minus = Ti_min_halves[i]
    pi_current = -(qsc[i] + (factor * pi[i]))

    pi_next_plus_arr.append(float(pi_next_plus))
    pi_next_arr.append(float(pi_next))
    pi_next_minus_arr.append(float(pi_next_minus))
    pi_arr.append(float(pi_current))

  pi_next_plus_arr = pi_next_plus_arr[:-1]
  pi_next_minus_arr = pi_next_minus_arr[1:]

  p_new = TDMAsolver(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr, pi_arr)
  p_new_arr.append(p_new)
  pi = p_new # updating pi

df = pd.DataFrame.from_records(p_new_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5
0,15.0,5999.082535,5995.023025,5968.941865,5805.437607,5964.134016
1,30.0,5996.288586,5983.926184,5922.463533,5655.347957,5907.211835
2,45.0,5990.905854,5967.088850,5868.771602,5532.883385,5838.207451
3,60.0,5982.510213,5945.361975,5812.078717,5427.946146,5762.579140
4,75.0,5970.918387,5919.627971,5754.472173,5334.424389,5683.652318
5,90.0,5956.120544,5890.644450,5696.924816,5248.575083,5603.449393
6,105.0,5938.219314,5859.011646,5639.838141,5168.072925,5523.191307
7,120.0,5917.382850,5825.187636,5583.331141,5091.447822,5443.602932
8,135.0,5893.811872,5789.517243,5527.391637,5017.750206,5365.100470
9,150.0,5867.718063,5752.260646,5471.953765,4946.350635,5287.907367


## Example 5.12 Similar to Ex 5.8, but **constant pressure at left boundary of reservoir**

![Example 5 9](https://user-images.githubusercontent.com/51282928/76109912-081bbc00-6010-11ea-9f00-023e9d954ac6.PNG)


Modification:

* For gridblock 1, $p_{i}^{n+1}=6000$ (fixed), so it's automatically out from the matrix equation

* For gridblock 2, the implicit equation becomes:
$$(0.1268) \cdot p_{i+1}^{n+1}-(0.8146) \cdot p_{i}^{n+1} + (0.1268) \cdot (6000) = -3665.98$$because $p_{i-1}^{n+1} = 6000$ (fixed)

* So, we will only have matrix ($4x4$) to solve.

$$A \cdot X = B$$

Where, $A=\begin{bmatrix} coeff(p_{2}^{n}) & coeff(p_{3}^{n}) & 0 & 0 \\ coeff(p_{2}^{n}) & coeff(p_{3}^{n}) & coeff(p_{4}^{n}) & 0 & \\ 0 & coeff(p_{3}^{n}) & coeff(p_{4}^{n}) & coeff(p_{5}^{n}) \\ 0 & 0 & coeff(p_{4}^{n}) & coeff(p_{5}^{n}) \end{bmatrix} = \begin{bmatrix} -0.81 & 0.13 & 0 & 0 \\ 0.13 & -0.81 & 0.13 & 0  \\ 0 & 0.13 & -0.81 & 0.13 \\ 0 & 0 & 0.13 & -0.69 \end{bmatrix}$

$X=\begin{bmatrix} p_{1}^{n+1} \\ p_{2}^{n+1} \\ p_{3}^{n+1} \\ p_{4}^{n+1}\end{bmatrix}$, and $B=\begin{bmatrix} -4126.78 \\ -3365.98 \\ -3215.98 \\ -3365.98 \end{bmatrix}$

Modification is marked in the lines between `###`


### For the first timestep

In [18]:
import sys, os
sys.path.append('/content/reservoir-engineering/Reservoir Simulation Ertekin/Unit 5 Finite-Difference Approximation to Linear-Flow Equations')
from tdma_implicit import TDMAsolver

pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]

pi_next_plus_arr = []
pi_next_arr = []
pi_next_minus_arr = []
pi_arr = []

###
for i in range(1,len(pi)):
  if i == 1:
    pi_next_plus = Ti_plus_halves[i]
    pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
    pi_next_minus = 0
    pi_current = -(qsc[i] + (factor * pi[i])) - ((Ti_min_halves[i]) * p_initial)
  if i > 1:
    pi_next_plus = Ti_plus_halves[i]
    pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
    pi_next_minus = Ti_min_halves[i]
    pi_current = -(qsc[i] + (factor * pi[i]))

  pi_next_plus_arr.append(float(pi_next_plus))
  pi_next_arr.append(float(pi_next))
  pi_next_minus_arr.append(float(pi_next_minus))
  pi_arr.append(float(pi_current))

pi_next_plus_arr = pi_next_plus_arr[:-1]
pi_next_minus_arr = pi_next_minus_arr[1:]
###

p_new = TDMAsolver(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr, pi_arr)
p_new = np.append([p_initial], p_new)
p_new

array([6000.        , 5995.16946579, 5968.96524107, 5805.44135273,
       5964.13470687])

In [19]:
# form of matrix
def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

A = tridiag(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr)
A

array([[-0.81457233,  0.1267875 ,  0.        ,  0.        ],
       [ 0.1267875 , -0.81457233,  0.1267875 ,  0.        ],
       [ 0.        ,  0.1267875 , -0.81457233,  0.1267875 ],
       [ 0.        ,  0.        ,  0.1267875 , -0.68778483]])

### For the next timesteps (iterative)

In [10]:
import sys, os
sys.path.append('/content/reservoir-engineering/Reservoir Simulation Ertekin/Unit 5 Finite-Difference Approximation to Linear-Flow Equations')
from tdma_implicit import TDMAsolver

pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

p_new_arr = []

for j in range(len(time)):
  pi_next_plus_arr = []
  pi_next_arr = []
  pi_next_minus_arr = []
  pi_arr = []

  ###
  for i in range(1,len(pi)):
    if i == 1:
      pi_next_plus = Ti_plus_halves[i]
      pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
      pi_next_minus = 0
      pi_current = -(qsc[i] + (factor * pi[i])) - ((Ti_min_halves[i]) * p_initial)
    if i > 1:
      pi_next_plus = Ti_plus_halves[i]
      pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
      pi_next_minus = Ti_min_halves[i]
      pi_current = -(qsc[i] + (factor * pi[i]))

    pi_next_plus_arr.append(float(pi_next_plus))
    pi_next_arr.append(float(pi_next))
    pi_next_minus_arr.append(float(pi_next_minus))
    pi_arr.append(float(pi_current))

  pi_next_plus_arr = pi_next_plus_arr[:-1]
  pi_next_minus_arr = pi_next_minus_arr[1:]
  ###

  p_new = TDMAsolver(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr, pi_arr)
  p_new = np.append([p_initial], p_new)
  p_new_arr.append(p_new)
  pi = p_new # updating pi

df = pd.DataFrame.from_records(p_new_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5
0,15.0,6000.0,5995.169466,5968.965241,5805.441353,5964.134707
1,30.0,6000.0,5984.624711,5922.591989,5655.371288,5907.216699
2,45.0,6000.0,5969.048666,5869.177922,5532.965675,5838.226588
3,60.0,6000.0,5949.585085,5813.049560,5428.162570,5762.634645
4,75.0,6000.0,5927.365949,5756.418771,5334.897035,5683.784720
5,90.0,6000.0,5903.341650,5700.382857,5249.481657,5603.724507
6,105.0,6000.0,5878.247313,5645.459576,5169.652506,5523.706889
7,120.0,6000.0,5852.623762,5591.870859,5094.003673,5444.494621
8,135.0,6000.0,5826.855115,5539.691563,5021.649999,5366.546678
9,150.0,6000.0,5801.206634,5488.926293,4952.024595,5290.132927


## Example 5.13 Halving the grid spacing of Ex 5.9 (more refined grids)

![Example 5 10](https://user-images.githubusercontent.com/51282928/76109917-0c47d980-6010-11ea-95bf-87492c658368.PNG)

In [20]:
# known
p_initial = 6000 # initial pressure, in psia
delta_x = 500 # ft, half the previous one
delta_y = 1000
delta_z = 75
ngrid = 10 # now twice the number of previous gridblocks
grid_loc = 4 # grid location where production well is located
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 3.5E-06 # phase compressibility, psi^-1
k_x = 15 # perm in x direction, md
poro = 0.18
mu = 10 # phase viscosity, cp

delta_t = 15 # days
qsc = -150 # minus means production, stb/d

# conversion
k_x = 15 * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (Vb * poro * c) / (alpha * B * delta_t)
factor

0.28049866429207476

In [21]:
# calculate transmissibility of coupling cells
beta = 1.127 # transmissibility conversion factor, is a constant
Ax = delta_y * delta_z
T_plus_half = beta * ((Ax * k_x) / (mu * B * delta_x))
T_min_half = T_plus_half
T_min_half, T_plus_half

(0.253575, 0.253575)

In [22]:
q = np.full(ngrid-1, T_min_half)
Ti_plus_halves = np.append(q, [0])
print(Ti_plus_halves)
p = np.full(ngrid-1, T_plus_half)
Ti_min_halves = np.append([0], p)
print(Ti_min_halves)
print("At grid 1, the coupling transmissibility coeffs are:", Ti_min_halves[0], "for T_min_half and:", Ti_plus_halves[0], "for T_plus_half.")
print("At grid 3, the coupling transmissibility coeffs are:", Ti_min_halves[2], "for T_min_half and:", Ti_plus_halves[2], "for T_plus_half.")
print("At grid 5, the coupling transmissibility coeffs are:", Ti_min_halves[4], "for T_min_half and:", Ti_plus_halves[4], "for T_plus_half.")

[0.253575 0.253575 0.253575 0.253575 0.253575 0.253575 0.253575 0.253575
 0.253575 0.      ]
[0.       0.253575 0.253575 0.253575 0.253575 0.253575 0.253575 0.253575
 0.253575 0.253575]
At grid 1, the coupling transmissibility coeffs are: 0.0 for T_min_half and: 0.253575 for T_plus_half.
At grid 3, the coupling transmissibility coeffs are: 0.253575 for T_min_half and: 0.253575 for T_plus_half.
At grid 5, the coupling transmissibility coeffs are: 0.253575 for T_min_half and: 0.253575 for T_plus_half.


In [0]:
qsc = [0, 0, 0, 0, 0, 0, -75, -75, 0, 0] # production well in grid 7 and 8

In [28]:
import sys, os
sys.path.append('/content/reservoir-engineering/Reservoir Simulation Ertekin/Unit 5 Finite-Difference Approximation to Linear-Flow Equations')
from tdma_implicit import TDMAsolver

pi = np.full(ngrid, p_initial) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(15, 370, delta_t)

p_new_arr = []

for j in range(len(time)):
  pi_next_plus_arr = []
  pi_next_arr = []
  pi_next_minus_arr = []
  pi_arr = []

  ###
  for i in range(1,len(pi)):
    if i == 1:
      pi_next_plus = Ti_plus_halves[i]
      pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
      pi_next_minus = 0
      pi_current = -(qsc[i] + (factor * pi[i])) - ((Ti_min_halves[i]) * p_initial)
    if i > 1:
      pi_next_plus = Ti_plus_halves[i]
      pi_next = -(factor + Ti_plus_halves[i] + Ti_min_halves[i])
      pi_next_minus = Ti_min_halves[i]
      pi_current = -(qsc[i] + (factor * pi[i]))

    pi_next_plus_arr.append(float(pi_next_plus))
    pi_next_arr.append(float(pi_next))
    pi_next_minus_arr.append(float(pi_next_minus))
    pi_arr.append(float(pi_current))

  pi_next_plus_arr = pi_next_plus_arr[:-1]
  pi_next_minus_arr = pi_next_minus_arr[1:]
  ###

  p_new = TDMAsolver(pi_next_minus_arr, pi_next_arr, pi_next_plus_arr, pi_arr)
  p_new = np.append([p_initial], p_new)
  p_new_arr.append(p_new)
  pi = p_new # updating pi

df = pd.DataFrame.from_records(p_new_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5', 'grid 6', 'grid 7', 'grid 8', 'grid 9', 'grid 10'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5,grid 6,grid 7,grid 8,grid 9,grid 10
0,15.0,6000.0,5999.046754,5997.039050,5991.756013,5977.353672,5937.900500,5829.754330,5829.056910,5935.036771,5969.155845
1,30.0,6000.0,5996.259834,5989.436844,5974.204473,5939.557003,5863.099752,5703.899684,5701.252668,5853.002107,5914.006741
2,45.0,6000.0,5991.309297,5977.142427,5949.375765,5894.144174,5788.677985,5600.888200,5594.920031,5767.099697,5844.256150
3,60.0,6000.0,5984.334088,5960.952365,5919.661577,5845.501657,5717.534538,5510.870219,5500.400623,5681.146376,5766.812599
4,75.0,6000.0,5975.710210,5941.880891,5886.955159,5795.850360,5649.822472,5428.893100,5413.053060,5596.361695,5685.883523
5,90.0,6000.0,5965.868725,5920.851032,5852.570607,5746.254804,5585.077571,5352.281061,5330.508608,5513.197918,5603.893425
6,105.0,6000.0,5955.198856,5898.594955,5817.371909,5697.212897,5522.804888,5279.512310,5251.495144,5431.845635,5522.206157
7,120.0,6000.0,5944.013149,5875.652933,5781.914795,5648.954837,5462.612953,5209.688177,5175.296079,5352.384861,5441.576016
8,135.0,6000.0,5932.545920,5852.407064,5746.554178,5601.586214,5404.220200,5142.261505,5101.487504,5274.838686,5362.410123
9,150.0,6000.0,5920.965319,5829.120451,5711.516483,5555.154646,5347.431306,5076.889436,5029.805011,5199.197592,5284.917784
